In [1]:
"""Kleine todo:
    -goed kijken co2 data zweden want die is wel erg laag
    -kaart animatie groter krijgen
    -misschien iets doen met gdp / capita ?
    -toevoegen van hoofd- en deelvragen
    - opschonen (er wordt nog vel onzin dingen geprint)
    - misschien ook een kaart maken van zev per land en vergelijken met de gdp / capita kaart (deelvraag 3)"""

# Importing the necessary libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [13]:
# Load database with number of passenger vehicles per country per year
database_1 = pd.read_csv('data/road_eqs_carpda_page_linear.csv')

# Load database with number of zero emmision vehicles per country per year
database_2 = pd.read_csv('data/road_eqs_zev_page_linear.csv')

# Load database with emmisions per vehicles per country per year
database_3 = pd.read_csv('data/sdg_13_10_page_linear_2.csv')

database_4 = pd.read_csv('data/population_1january.csv')

In [25]:
database_4.columns

database_4 = database_4[['geo','TIME_PERIOD', 'OBS_VALUE']]
database_4 = database_4.rename(columns={'OBS_VALUE':'population'})


In [26]:
# Store the unique countries from each database in sets
set1 = set(database_1.geo.unique())
set2 = set(database_2.geo.unique())
set3 = set(database_3.geo.unique())
set4 = set(database_4.geo.unique())

In [27]:
# Identify which countries are in all three databases
countries = set1.intersection(set2, set3, set4)

In [28]:
joined_df_1 = pd.merge(database_1, database_2, how='right', on=['geo', 'TIME_PERIOD'])
joined_df_2 = pd.merge(joined_df_1, database_3, how='left', on=['geo', 'TIME_PERIOD'])
joined_df = pd.merge(joined_df_2, database_4, how='left', on=['geo', 'TIME_PERIOD'])

In [30]:
# Rename columns
joined_df = joined_df.rename(columns={'OBS_VALUE_y':'amount_zev', 'OBS_VALUE' : 'emission_per_capita', 'OBS_VALUE_x' : 'amount_passenger_cars', 'TIME_PERIOD' : 'time_period'})
#joined_df['amount_zev'].head()

0     3389
1     5038
2     9086
3    14637
4    20855
Name: amount_zev, dtype: int64

In [324]:
# Transforming the abbrevation to the full country names
countries_map = {'AT' : 'Austria',
 'BE' : 'Belgium',
 'BG' : 'Bulgaria',
 'CY' : 'Cyprus',
 'CZ' : 'Czech Republic',
 'DE' : 'Germany',
 'DK': 'Denmark',
 'EE' : 'Estonia',
 'EL' : 'Greece',
 'ES' : 'Spain',
 'EU27_2020' : 'European Union',
 'FI' : 'Finland',
 'FR' : 'France',
 'HR' : 'Croatia',
 'HU' : 'Hungary',
 'IE' :'Ireland',
 'IT' : 'Italy',
 'LT' : 'Lithuania',
 'LU' : 'Luxembourg',
 'LV' : 'Latvia',
 'MT' : 'Malta',
 'NL' : 'Netherlands',
 'PL' : 'Poland',
 'PT': 'Portugal',
 'RO' : 'Romania',
 'SE' : 'Sweden',
 'SI' : 'Slovenia',
 'SK' : 'Slovakia'}

joined_df['geo'] = joined_df['geo'].map(countries_map)

In [325]:
# Construct new columns with share of zero emmission vehicles and amount of emmission vehicles
joined_df['share_zev'] = joined_df['amount_zev'] / joined_df['amount_passenger_cars']
joined_df['amount_ev'] = joined_df['amount_passenger_cars'] - joined_df['amount_zev']

In [327]:
# Select only the relevant columns from the merged dataset
joined_df = joined_df[['geo', 'time_period', 'amount_passenger_cars', 'amount_zev', 'amount_ev', 'share_zev', 'emission_per_capita']]
joined_df

,geo,time_period,amount_passenger_cars,amount_zev,amount_ev,share_zev,emission_per_capita
0,Austria,2014,4694921,3389,4691532,0.000722,8.4
1,Austria,2015,4748048,5038,4743010,0.001061,8.7
2,Austria,2016,4821557,9086,4812471,0.001884,8.6
3,Austria,2017,4898578,14637,4883941,0.002988,9.3
4,Austria,2018,4978852,20855,4957997,0.004189,9.8
...,...,...,...,...,...,...,...
275,Slovakia,2019,2393577,956,2392621,0.000399,6.4
276,Slovakia,2020,2439986,1863,2438123,0.000764,5.5
277,Slovakia,2021,2493183,3001,2490182,0.001204,6.3
278,Slovakia,2022,2555491,4531,2550960,0.001773,5.5


In [328]:
# Check whether there are missing values
rows_with_na = joined_df[joined_df.isna().any(axis=1)]
rows_with_na

,geo,time_period,amount_passenger_cars,amount_zev,amount_ev,share_zev,emission_per_capita
9,Austria,2023,5185006,155557,5029449,0.030001,NaN
19,Belgium,2023,6047551,181476,5866075,0.030008,NaN
29,Bulgaria,2023,3006215,11473,2994742,0.003816,NaN
39,Cyprus,2023,621116,1253,619863,0.002017,NaN
49,Czech Republic,2023,6512774,22469,6490305,0.003450,NaN
59,Germany,2023,49098685,1410887,47687798,0.028736,NaN
69,Denmark,2023,2827864,200152,2627712,0.070779,NaN
79,Estonia,2023,865773,5799,859974,0.006698,NaN
89,Greece,2023,5877759,12315,5865444,0.002095,NaN
99,Spain,2023,26778142,150315,26627827,0.005613,NaN


In [7]:
database_4.head(30)


,DATAFLOW,LAST UPDATE,freq,indic_de,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2013,1.9,NaN
1,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2014,1.9,NaN
2,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2015,1.9,NaN
3,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2016,2.0,NaN
4,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2017,2.0,NaN
5,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2018,2.0,p
6,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2019,2.0,be
7,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2020,2.0,e
8,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2021,2.0,ep
9,ESTAT:TPS00005(1.0),17/10/24 23:00:00,A,POPSHARE_EU27_2020,AT,2022,2.0,bep


# Deelvraag 1

In [329]:
# Group databases by country
groups = joined_df.groupby('geo')

In [330]:
# Plots the amount of ZEV against the total amount of vehicles. NB the values on the y-axes are scaled according to the data
for name, group in groups:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=group['time_period'], y=group['amount_zev'],
                             mode='lines+markers',
                             name='Amount ZEV',
                             yaxis='y1'))
    fig.add_trace(go.Scatter(x=group['time_period'], y=group['amount_ev'],
                             mode='lines+markers',
                             name='Amount EV',
                             yaxis='y2'))

    fig.update_layout(
        title=f'Vehicle Data for {name}',
        xaxis_title='Time Period',
        yaxis_title='Amount ZEV',
        yaxis=dict(
            title='Amount ZEV',
            side='left',
            showgrid=False
        ),
        yaxis2=dict(
            title='Amount EV',
            side='right',
            overlaying='y',
            showgrid=False
        ),
        legend_title='Legend'
    )

    fig.show()

# Deelvraag 2

In [331]:
# Plot the share of ZEV against the greenhouse emmision per capita for each country
for name, group in groups:
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=group['time_period'], y=group['emission_per_capita'],
                             mode='lines+markers',
                             name='Emission per Capita',
                             yaxis='y1'))

    fig.add_trace(go.Scatter(x=group['time_period'], y=group['share_zev'],
                             mode='lines+markers',
                             name='Share ZEV',
                             yaxis='y2'))

    fig.update_layout(
        title=f'Environmental Data for {name}',
        xaxis_title='Time Period',
        yaxis=dict(
            title='Emission per Capita',
            side='left',
            showgrid=False,
            type='linear'
        ),
        yaxis2=dict(
            title='Share ZEV',
            side='right',
            overlaying='y',
            showgrid=False,
            type='linear'
        ),
        legend_title='Legend'
    )


    fig.show()

# Deelvraag 3

In [332]:
# Making a plot with all the countries and their share of ZEV's
fig = go.Figure()

for name, group in groups:
    fig.add_trace(go.Scatter(
        x=group['time_period'],
        y=group['share_zev'],
        mode='lines+markers',
        name=name
    ))

fig.update_layout(
    title='Comparison of Share ZEV over Time by Geo Group',
    xaxis_title='Time Period',
    yaxis_title='Share ZEV',
    legend_title='Geo Group'
)

fig.show()

In [335]:
joined_df.time_period.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [336]:
# Filtering out the NaNs of emmission data
filtered_df = joined_df[joined_df['time_period'] != 2023]

#  Making a map animation wich shows the emmissions per EU-country over time
fig = px.choropleth(filtered_df,
                    locations='geo',
                    locationmode='country names',
                    color='emission_per_capita',
                    color_continuous_scale='Viridis',
                    scope='europe',
                    title='Emissions per EU-country over time',
                    animation_frame='time_period',
                    range_color=[0, 16]
                   )

fig.update_layout(
    width=1000,
    height=600,
    margin={"r":0,"t":50,"l":0,"b":0},
    title_x=0.5
    )

fig.show()